In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:77% !important; }</style>"))
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [20]:
import os
import re
import pickle
import numpy as np
np.set_printoptions(linewidth=100)

In [2]:
graph_args = {'layout':'openpose', 'strategy':'spatial'}

In [15]:
data_dir_path = '/root/sharedfolder/datasets/data_ssd/kinetics-skeleton/st-gcn_kinetics/Kinetics/kinetics-skeleton/'
train_data_path = os.path.join(data_dir_path, 'train_data.npy')
train_label_path = os.path.join(data_dir_path, 'train_label.pkl')
test_data_path = os.path.join(data_dir_path, 'val_data.npy')
test_label_path = os.path.join(data_dir_path, 'val_label.pkl')

In [16]:
train_npy = np.load(train_data_path, mmap_mode='r')
test_npy = np.load(test_data_path, mmap_mode='r')

In [17]:
save_files = False
gen_train = False
if gen_train:
    split_npy = train_npy
    split_2gen = 'normal_train'
    f = train_label_path
else:
    split_npy = test_npy
    split_2gen = 'normal_test'
    f = test_label_path

In [22]:
sample_name, label = pickle.load(open(test_label_path, "rb"))

EOFError: Ran out of input

In [8]:
data_path = dict()
label_path = dict()
split_indices = dict()
data_npy = dict()
data_pkl = dict()  # An array of [sample_name, label]

In [10]:
# Train Split Generator
data_pkl['normal_train'] = [[],[]]  # Important to zero so not to accumulate older values
data_pkl['normal_test'] = [[],[]]
for split in ['front_raises134', 'deadlifting88', 'clean_jerk59', 'forcast254', 'normal_train', 'normal_test', 'abnormal_test']:
    data_path[split]  =  os.path.join(data_dir_path, 'ad_experiment', split+'_data.npy')
    label_path[split] =  os.path.join(data_dir_path, 'ad_experiment', split+'_label.pkl')
    split_num = re.findall('\d+', split)
    if split_num != []:
        split_indices[split] = [ind for ind, val in enumerate(label) if val == int(split_num[0])]
        data_npy[split] = split_npy[split_indices[split]]
        data_pkl[split] = [[sample_name[i] for i in split_indices[split]], [label[i] for i in split_indices[split]]]
        data_pkl[split_2gen][0] += data_pkl[split][0]
        data_pkl[split_2gen][1] += data_pkl[split][1]
        print("Split {}, data_pkl[split][0] len={}, data_pkl[split_2gen][0] len={}".format(split, len(data_pkl[split][0]), len(data_pkl[split_2gen][0])))

data_npy[split_2gen] = np.concatenate((data_npy['front_raises134'], data_npy['deadlifting88'], data_npy['clean_jerk59'], data_npy['forcast254']))

NameError: name 'label' is not defined

In [ ]:
if save_files and gen_train:
    np.save(data_path['normal_train'], data_npy['normal_train'])
    pickle.dump(data_pkl['normal_train'], open(label_path['normal_train'] ,"wb"))
elif save_files:
    np.save(data_path['normal_test'], data_npy['normal_test'])
    pickle.dump(data_pkl['normal_test'], open(label_path['normal_test'] ,"wb"))